In [2]:
import sys
import math
import pickle
import numpy as np

k = 0.1
n = 3
m = 2

sys.setrecursionlimit(1500)

In [3]:
def inputDigit(name="digitdata/trainingimages"):
    with open(name) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
    return image

In [4]:
digitImage = inputDigit()

In [5]:
import collections

def readRough(name="digitdata/traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels


In [6]:
label = readRough()
print (len(label))
#print sorted(collections.Counter(label).items())
class_ = sorted(collections.Counter(label).items())
#print (class_)
with open('labelstats.txt', 'wb') as fp:
    pickle.dump(class_, fp)

5000


In [7]:
with open ('labelstats.txt', 'rb') as fp:
    class_ = pickle.load(fp)

digitImage = inputDigit()
training=np.zeros(shape=(10, 28-n+1, 28-m+1, 2**(n*m)))

count = 0
for i in range(5000):                  # for each image
    for row in range(28-n+1):
        for col in range(28-m+1):      # for each feature
            
            feature_val = 0
            for y in range(n):
                for x in range(m):      # for each pixel in feature (overlapping)
                    # calculate feature_val
                    if digitImage[i*28+row+y][col+x] != ' ':
                        feature_val += 2**(m*y+x)
            training[int(label[i])][row][col][feature_val] += 1

for i in range(10):
    training[i] = (training[i] + k) / (class_[i][1] + k * 2**(m*n))

#print(training[0][0].shape)
print(training[0][0])



[[  9.87021014e-01   2.06015657e-04   2.06015657e-04 ...,   2.06015657e-04
    2.06015657e-04   2.06015657e-04]
 [  9.87021014e-01   2.06015657e-04   2.06015657e-04 ...,   2.06015657e-04
    2.06015657e-04   2.06015657e-04]
 [  9.87021014e-01   2.06015657e-04   2.06015657e-04 ...,   2.06015657e-04
    2.06015657e-04   2.06015657e-04]
 ..., 
 [  9.87021014e-01   2.06015657e-04   2.06015657e-04 ...,   2.06015657e-04
    2.06015657e-04   2.06015657e-04]
 [  9.87021014e-01   2.06015657e-04   2.06015657e-04 ...,   2.06015657e-04
    2.06015657e-04   2.06015657e-04]
 [  9.87021014e-01   2.06015657e-04   2.06015657e-04 ...,   2.06015657e-04
    2.06015657e-04   2.06015657e-04]]


In [10]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

test_rough = inputDigit(name = "digitdata/testimages")
answer = np.zeros(1000)


for i in range(1000):                  # for each image
    test_image = np.zeros(shape=(28-n+1, 28-m+1))
    for row in range(28-n+1):
        for col in range(28-m+1):      # for each feature
            
            feature_val = 0
            for y in range(n):
                for x in range(m):      # for each pixel in feature (overlapping)
                    # calculate feature_val
                    if test_rough[i*28+row+y][col+x] != ' ':
                        feature_val += 2**(2*y+x)
            test_image[row][col] = feature_val
       
    posteriori = np.zeros(10)
    for class_num in range(10):
        posteriori[class_num] = math.log(class_[class_num][1])
        for row in range(28-n+1):
            for col in range(28-m+1):
                posteriori[class_num] += math.log(training[class_num][row][col][int(test_image[row][col])])
    answer[i] = np.argmax(posteriori)
    #print(posteriori)
    
print(answer)



[ 9.  0.  2.  3.  1.  9.  7.  8.  1.  0.  4.  1.  9.  9.  5.  4.  2.  6.
  8.  1.  3.  7.  9.  4.  4.  1.  8.  1.  3.  8.  1.  2.  8.  8.  0.  6.
  2.  1.  1.  9.  1.  5.  3.  4.  6.  9.  5.  0.  9.  2.  2.  4.  8.  2.
  1.  9.  2.  4.  9.  4.  4.  0.  9.  9.  2.  2.  3.  3.  8.  3.  5.  7.
  3.  5.  8.  1.  2.  4.  4.  6.  4.  9.  5.  1.  0.  6.  9.  5.  9.  5.
  9.  7.  3.  8.  0.  3.  7.  1.  3.  6.  7.  2.  5.  9.  7.  9.  6.  3.
  6.  3.  7.  4.  6.  5.  8.  5.  4.  7.  8.  8.  8.  0.  7.  6.  8.  9.
  7.  3.  7.  1.  9.  5.  2.  7.  3.  5.  1.  1.  2.  1.  4.  7.  4.  7.
  5.  4.  5.  4.  0.  8.  3.  6.  9.  5.  0.  2.  7.  4.  4.  4.  4.  6.
  6.  4.  9.  9.  3.  4.  5.  9.  8.  7.  3.  9.  2.  7.  0.  2.  4.  1.
  1.  1.  8.  9.  2.  8.  7.  2.  0.  1.  5.  0.  4.  1.  7.  0.  6.  0.
  8.  6.  5.  1.  8.  0.  9.  3.  8.  2.  3.  5.  2.  1.  6.  1.  1.  3.
  7.  9.  0.  3.  0.  5.  4.  0.  6.  8.  2.  2.  8.  8.  4.  0.  9.  5.
  5.  5.  1.  2.  1.  3.  1.  7.  9.  5.  7.  2.  0

In [11]:
testlabels = readRough("digitdata/testlabels")


In [12]:
def confusion_matrix():
    num_each_class = np.zeros(10)
    conf_matrix = np.zeros(shape=(10,10))
    for i in range(1000):
        conf_matrix[int(testlabels[i])][int(answer[i])] += 1
        num_each_class[int(testlabels[i])] += 1
            
    for i in range(10):
        for j in range(10):
            conf_matrix[i][j] /= num_each_class[i]
    
    return conf_matrix

In [13]:
conf_matrix = confusion_matrix()
conf_matrix = np.around(conf_matrix, 2)
for row in conf_matrix:
    print(row)

[ 0.98  0.    0.    0.    0.    0.    0.01  0.    0.01  0.  ]
[ 0.    0.98  0.    0.    0.01  0.    0.01  0.    0.    0.  ]
[ 0.    0.01  0.9   0.02  0.01  0.    0.02  0.01  0.03  0.  ]
[ 0.    0.    0.    0.93  0.    0.01  0.    0.03  0.01  0.02]
[ 0.    0.    0.    0.    0.94  0.    0.02  0.01  0.    0.03]
[ 0.01  0.    0.    0.09  0.    0.85  0.    0.01  0.02  0.02]
[ 0.01  0.01  0.    0.    0.    0.05  0.91  0.    0.01  0.  ]
[ 0.    0.04  0.04  0.    0.02  0.    0.    0.79  0.02  0.09]
[ 0.01  0.01  0.03  0.08  0.    0.02  0.    0.01  0.83  0.02]
[ 0.01  0.    0.    0.03  0.03  0.01  0.    0.01  0.02  0.89]
